In [13]:
import rdflib
from rdflib.graph import Graph, URIRef
from SPARQLWrapper import SPARQLWrapper, XML


In [14]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

construct_query= """
PREFIX ma: <http://www.semanticweb.org/dibah/ontologies/2024/3/untitled-ontology-11#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX wd: <http://www.wikidata.org/entity/>

CONSTRUCT {
    ?book rdf:type ma:Book .
    ?book ma:title ?name .
    ?book ma:written_by ?author .
    ?author rdf:type ma:Author .
    ?book ma:published_by ?publisher .
    ?publisher rdf:type ma:publisher .
    ?book ma:has_genre ?genre .
    ?genre rdf:type ma:literaryGenre .
    ?book ma:cover_designed_by ?coverArtist .
    ?book ma:numberOfPages ?numberOfPages .
    ?book ma:isbn ?isbn .
    ?book ma:releaseDate ?releaseDate .
    ?book ma:has_publish_country ?country .
    ?country rdf:type ma:Country .
    ?book ma:has_language ?lang .
    ?lang rdf:type ma:Language .
    ?book ma:has_character ?Character .
    ?Character rdf:type ma:Character .      
  }
WHERE {
  ?book wdt:P31 wd:Q571 .
  ?book wdt:P1476 ?name .
  OPTIONAL { ?book wdt:P407 ?lang .}
  OPTIONAL { ?book wdt:P50 ?author . }
  OPTIONAL { ?book wdt:P136 ?genre . }
  OPTIONAL { ?book wdt:P123 ?publisher . }
  ?book wdt:P577 ?releaseDate .
  OPTIONAL { ?book wdt:P495 ?Country . }
  OPTIONAL { ?book wdt:P736 ?coverArtist . }
  OPTIONAL { ?book wdt:P674 ?Character . }

  FILTER (LANG(?name) = 'en') .
}"""

sparql.setQuery(construct_query)
sparql.setReturnFormat(XML)

graph_id = URIRef("http://www.semanticweb.org/store/book")
g = Graph(identifier=graph_id)

print("I might take some time, bear with me...")

g = sparql.query().convert()

g.parse("book_dbpedia.owl")

g.serialize("book_bonus.owl", format="xml")

print("...All done!")
print("")

I might take some time, bear with me...
...All done!



In [21]:
author_query_template = """
SELECT DISTINCT ?author ?birthdate ?birthplace ?gender WHERE {{
    BIND(<{}> AS ?author).
    ?author wdt:P569 ?birthdate .
    OPTIONAL {{ ?author wdt:P19 ?birthplace . }}
    OPTIONAL {{ ?author wdt:P21 ?gender . }}
}}
"""
author_query_template.format("b")

'\nSELECT DISTINCT ?author ?birthdate ?birthplace ?gender WHERE {\n    BIND(<b> AS ?author).\n    ?author wdt:P569 ?birthdate .\n    OPTIONAL { ?author wdt:P19 ?birthplace . }\n    OPTIONAL { ?author wdt:P21 ?gender . }\n}\n'

In [29]:
import rdflib
from rdflib.graph import Graph, URIRef
from SPARQLWrapper import SPARQLWrapper, XML, JSON
from rdflib.namespace import RDF

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
ma = URIRef("http://www.semanticweb.org/dibah/ontologies/2024/3/untitled-ontology-11#")
# Load the existing ontology file
g = Graph()
g.parse("book_dbpedia.owl")
author_query_template = """
SELECT DISTINCT ?author ?birthdate ?birthplace ?gender WHERE {{
    BIND(<{}> AS ?author).
    ?author wdt:P569 ?birthdate .
    OPTIONAL {{ ?author wdt:P19 ?birthplace . }}
    OPTIONAL {{ ?author wdt:P21 ?gender . }}
}}
"""
author_uris = set()
for subject, predicate, obj in g.triples((None, RDF.type, None)):
    for s, p, o in g.triples((subject, None, None)):
        if str(p).endswith("written_by"):
            author_uris.add(str(o))
for author_uri in author_uris:
    # Construct and execute SPARQL query for each author
    author_query = author_query_template.format(author_uri)
    sparql.setQuery(author_query)
    sparql.setReturnFormat(JSON)
    author_data = sparql.query().convert()
    print(author_data)
    for result in author_data['results']['bindings']:
        birthdate = result.get("birthdate")
        birthplace = result.get("birthplace")
        gender = result.get("gender")

    if birthdate:
        g.add((URIRef(author_uri), rdflib.URIRef("http://www.semanticweb.org/dibah/ontologies/2024/3/untitled-ontology-11#AuthorBirthDate"), rdflib.Literal(birthdate.value)))
    if birthplace:
        g.add((URIRef(author_uri), rdflib.URIRef("http://www.semanticweb.org/dibah/ontologies/2024/3/untitled-ontology-11#AuthorBirthPlace"), rdflib.URIRef(birthplace.value)))
    if gender:
        g.add((URIRef(author_uri), rdflib.URIRef("http://www.semanticweb.org/dibah/ontologies/2024/3/untitled-ontology-11#AuthorGender"), rdflib.URIRef(gender.value)))

# Serialize the modified graph to OWL format
g.serialize("book_bonus.owl", format="xml")

print("...All done!")
# for author in author_uris:
#     print(author)

{'head': {'vars': ['author', 'birthdate', 'birthplace', 'gender']}, 'results': {'bindings': []}}


NameError: name 'birthdate' is not defined

In [6]:
author_uris

{'http://dbpedia.org/resource/Adrian_Hyland',
 'http://dbpedia.org/resource/Alberto_Vázquez-Figueroa',
 'http://dbpedia.org/resource/Alex_Haley',
 'http://dbpedia.org/resource/Alex_Miller_(writer)',
 'http://dbpedia.org/resource/Alice_Borchardt',
 'http://dbpedia.org/resource/Amelia_Atwater-Rhodes',
 'http://dbpedia.org/resource/Amos_Oz',
 'http://dbpedia.org/resource/Amélie_Nothomb',
 'http://dbpedia.org/resource/Anaïs_Nin',
 'http://dbpedia.org/resource/Andrea_Camilleri',
 'http://dbpedia.org/resource/Andreas_Norman',
 'http://dbpedia.org/resource/Andrew_Clements',
 'http://dbpedia.org/resource/Andrey_Kurkov',
 'http://dbpedia.org/resource/Andy_Griffiths_(author)',
 'http://dbpedia.org/resource/Anita_Shreve',
 'http://dbpedia.org/resource/Ann_M._Martin',
 'http://dbpedia.org/resource/Anne_Frank',
 'http://dbpedia.org/resource/Anne_Holm',
 'http://dbpedia.org/resource/Anne_Rivers_Siddons',
 'http://dbpedia.org/resource/Antonella_Gambotto-Burke',
 'http://dbpedia.org/resource/Arnon_Gru

In [22]:
import rdflib
from rdflib.graph import Graph, URIRef
from SPARQLWrapper import SPARQLWrapper, XML

# Configuring the end-point and constructing query.
# Notice the various SPARQL constructs we are making use of:
#
#   * PREFIX to bind prefixes in our query
#   * CONSTRUCT to build new individuals from our query
#   * OPTIONAL to indicate that some fields may not exist and that's OK
#   * FILTER to constrain our query in some way
#

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
construct_query="""PREFIX ma: <http://www.semanticweb.org/dibah/ontologies/2024/3/untitled-ontology-11#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>        
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>
    PREFIX dbpprop: <http://dbpedia.org/property/>
    PREFIX dbp: <http://dbpedia.org/property/>

    CONSTRUCT {
    ?book rdf:type ma:Book .
    ?book rdfs:label ?name .
    ?book ma:title ?name .
    ?book ma:written_by ?author .
    ?author rdf:type ma:Author .
    ?author rdfs:label ?Nauthor .
    ?book ma:published_by ?publisher .
    ?publisher rdf:type ma:publisher .
    ?book ma:has_genre ?genre .
    ?genre rdf:type ma:literaryGenre .
    ?book ma:cover_designed_by ?coverArtist .
    ?coverArtist rdf:type ma:coverArtist .
    ?coverArtist rdfs:label ?NcoverArtist .
    ?book ma:numberOfPages ?numberOfPages .
    ?book ma:isbn ?isbn .
    ?book ma:releaseDate ?releaseDate .
    ?book ma:has_publish_country ?country .
    ?country rdf:type ma:Country .
    ?book ma:has_language ?lang .
    ?lang rdf:type ma:Language .
    ?book ma:has_character ?Character .
    ?Character rdf:type ma:Character .        
    
    }
    WHERE{
        ?book rdf:type dbpedia-owl:Book .

        ?book dbp:name ?name .
        FILTER(!REGEX(?name, "Book", "i"))
        ?book dbpedia-owl:author ?author .
        ?author dbp:birthName ?Nauthor .
        OPTIONAL {?book dbpedia-owl:genre ?genre}
        OPTIONAL {?book dbpedia-owl:publisher ?publisher}
        OPTIONAL {?book dbpedia-owl:coverArtist ?coverArtist .
        ?coverArtist dbp:name ?NcoverArtist}
        OPTIONAL {?book dbpedia-owl:numberOfPages ?numberOfPages}
        ?book dbpedia-owl:isbn ?isbn .
        FILTER(?isbn != "") . # Ensure ISBN is not an empty string
        ?book dbp:releaseDate ?releaseDate .
        OPTIONAL {?book dbpedia-owl:country ?country}
        OPTIONAL {?book dbpedia-owl:language ?lang}
        OPTIONAL {?book dbpedia-owl:Character ?Character}

        FILTER(?country != ?lang) # Ensure country and language are different
        FILTER(LANG(?name)='en')
        }
    LIMIT 1000
    """

sparql.setQuery(construct_query)
sparql.setReturnFormat(XML)

# Creating the RDF store and graph
# We've seen something similar in the lab sheets before, Week 3. We're telling
# the rdflib library to create a new graph and store it in memory (so, temporarily).

graph_id = URIRef("http://www.semanticweb.org/store/book")
g = Graph(identifier = graph_id)

# SPARQL queries can take some time to run, especially if the query is particularly
# large and you're grabbing very many items. 
#
# While experimenting you may want to use the LIMIT construct in SPARQL to take
# only a couple of items, this way you can experiment with things without waiting
# ages for a query to complete.
print("  I might take some time, bear with  me...")

# merging results and saving the store
# The Week 4 lab showed us this, so we know that running the query will return a
# valid RDFlib graph.
g = sparql.query().convert()

# We also saw in Week 3 that we can parse files as valid RDFlib graphs too. When
# we do both of these things, they will be merged together.
g.parse("book.owl")

# You can open this file in protege and compare it to the existing `movie.owl`
# ontology to see what we did. You could also open this in a text editor and have
# a poke around that way.
author_query_template = """
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    
    SELECT DISTINCT ?author ?birthdate ?birthplace ?gender WHERE {
    ?author a <http://www.semanticweb.org/dibah/ontologies/2024/3/untitled-ontology-11#Author> .
    ?author wdt:P569 ?birthdate .
    OPTIONAL { ?author wdt:P19 ?birthplace . }
    OPTIONAL { ?author wdt:P21 ?gender . }
}
"""

# Execute SPARQL query to retrieve data about authors
sparql.setQuery(author_query_template)
sparql.setReturnFormat(XML)
author_data = sparql.query().convert()
g.serialize("bluh.owl", format="xml")
print("done with bluh")
print(author_data)
# Iterate through the query results and update the ontology
for result in (dict(row) for row in author_data):
    author_uri = result["author"].value
    birthdate = result.get("birthdate")
    birthplace = result.get("birthplace")
    gender = result.get("gender")

    if birthdate:
        g.add((URIRef(author_uri), rdflib.URIRef("http://www.semanticweb.org/dibah/ontologies/2024/3/untitled-ontology-11#AuthorBirthDate"), rdflib.Literal(birthdate.value)))
    if birthplace:
        g.add((URIRef(author_uri), rdflib.URIRef("http://www.semanticweb.org/dibah/ontologies/2024/3/untitled-ontology-11#AuthorBirthPlace"), rdflib.URIRef(birthplace.value)))
    if gender:
        g.add((URIRef(author_uri), rdflib.URIRef("http://www.semanticweb.org/dibah/ontologies/2024/3/untitled-ontology-11#AuthorGender"), rdflib.URIRef(gender.value)))

# Serialize the modified graph to OWL format
g.serialize("book_bonus.owl", format="xml")

  I might take some time, bear with  me...
done with bluh


TypeError: 'Document' object is not iterable

In [11]:
import rdflib
from rdflib.graph import Graph, URIRef
from SPARQLWrapper import SPARQLWrapper, XML

# Configuring the end-point and constructing query.
# Notice the various SPARQL constructs we are making use of:
#
#   * PREFIX to bind prefixes in our query
#   * CONSTRUCT to build new individuals from our query
#   * OPTIONAL to indicate that some fields may not exist and that's OK
#   * FILTER to constrain our query in some way
#

sparql = SPARQLWrapper("http://dbpedia.org/sparql")

graph_id = URIRef("http://www.semanticweb.org/store/book")
g = Graph(identifier=graph_id)
g.parse("book_dbpedia.owl")

set()

In [23]:
from rdflib import Graph, URIRef, Literal, Namespace

# Create a new RDF graph
g = Graph()

# Define namespaces for your ontology
ma = Namespace("http://www.semanticweb.org/dibah/ontologies/2024/3/untitled-ontology-11#")

# Define subjects and objects
book_uri = URIRef("http://example.com/book1")
author_uri = URIRef("http://example.com/author1")
title_literal = Literal("Sample Book Title")
author_name_literal = Literal("John Doe")
# Add triples to the graph
g.add((book_uri, ma.hasTitle, title_literal))  # Add a title property to the book
g.add((book_uri, ma.hasAuthor, author_uri))    # Add an author property to the book
g.add((author_uri, ma.hasName, author_name_literal))  # Add a name property to the author

# Serialize the graph to an RDF/XML file
g.serialize("ontology.owl", format="xml")

<Graph identifier=N53fe5171fdcf470e953d00c1e67cadfd (<class 'rdflib.graph.Graph'>)>